In [4]:
from climada.hazard import TCTracks
import sys
import os
from pathlib import Path
sys.path.insert(0, '../..')
from util import DATA_DIR

In [1]:

path = DATA_DIR / "tropical_cyclones/tracks"
def get_tracks_by_basin(basin='EP', n_tracks=0):
    """
    Read tropical cyclone tracks from the IBTrACS dataset, with a given number of synthetic tracks
    using thetc_tracks.calc_perturbed_trajectories() method

    Parameters:
        basin (str, optional): Name of the tropical cyclone genesis basin. Default is 'EP'.
        n_tracks (int or str, optional): Number of synthetic tracks to generate. If set to 0,
                                         no synthetic tracks are generated. Default is 10.

    Returns:
        None
    """
    year_range = (2000, 2022)
    nb_syn_tracks = int(n_tracks)
    tc_tracks = TCTracks()
    tc_tracks.read_ibtracs_netcdf(genesis_basin=basin, year_range=year_range)
    tc_tracks.equal_timestep(time_step_h=1)
    if nb_syn_tracks>0:
        tc_tracks.calc_perturbed_trajectories(nb_synth_tracks=nb_syn_tracks)
    path_basin = path /basin
    isExist = os.path.exists(path_basin)
    if not isExist:
        os.makedirs(path_basin)
    tc_tracks.write_netcdf(path_basin)

for basin in ["NI", "SI", "NA", "SP", "WP", "SA", "EP"]:
    get_tracks_by_basin(basin=basin)


2024-08-04 12:20:22,426 - climada.hazard.tc_tracks - WARNING - The use of TCTracks.read_ibtracs_netcdf is deprecated. Use TCTracks.from_ibtracs_netcdf instead.
2024-08-04 12:20:22,631 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2022-01-27 18:47:08 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /Users/szelie/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2024-08-04 12:20:43,033 - climada.hazard.tc_tracks - WARNING - The use of TCTracks.read_ibtracs_netcdf is deprecated. Use TCTracks.from_ibtracs_netcdf instead.
2024-08-04 12:20:43,093 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2022-01-27 18:47:08 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /Users/szelie/climada/data/IBTrACS.ALL.v04r00.nc a

In [1]:
import pandas as pd
import numpy as np
from climada.hazard import Hazard
import xarray as xr
from climada.hazard import Centroids

# get centroids from heatwaves:

# Extract years from the `time` dimension
year=2003
# Initialize an empty list to store the hazards

# Loop over each unique year
file = DATA_DIR / f"heatwave_days_era5_0.25/heatwaves_monthly/indicator_{year}.nc"
yearly_hw = xr.open_dataset(file)

yearly_hw = yearly_hw.rename({"month":"time"})
yearly_hw['longitude'] = yearly_hw['longitude'].where(yearly_hw['longitude'] <= 180, yearly_hw['longitude'] - 360)

# Convert it into a hazard
yearly_hazard = Hazard.from_xarray_raster(yearly_hw, hazard_type='hw', intensity_unit='days', intensity='heatwaves_days')

centroids = yearly_hazard.centroids 


In [5]:
from climada.hazard import Centroids, TropCyclone, TCTracks
import sys
import os
from climada.util.api_client import Client

basins_centroids = {'SI': (0, 140, -60, 0), 'SP': (100, -120, -60, 0),
                    'SA': (-120, 0, -60, 0), 'NA': (-120, 0, 0, 60),
                    'WP': (100, -180, 0, 60), 'NI': (0, -100, 0, 60),
                    'EP': (100, -70, 0, 60)}

OUT_FILE_NAME = 'tropical_cyclone_3600arcsec_genesis_{basin}_2000_2022.hdf5'

def main(centroids):
    """
    Create a TropCyclone hazard set based on TCTracks for a specific basin. Should be ran
    after compute_tc_tracks.py

    The function reads the TC tracks for the specified basin, calculates the hazard,
    and saves the TropCyclone hazard set to an output file.

    Parameters:
        basin (str, optional): Name of the TC basin. Default is 'EP'.
        n_tracks (int, optional): Number of synthetic tracks to consider. Default is 10.

    """
    path = DATA_DIR / "tropical_cyclones/tracks"

    path_basin = path /basin
    tc_tracks = TCTracks.from_netcdf(path_basin)
    centroids = centroids.select(extent=tc_tracks.get_extent(5))
    tc_haz = TropCyclone.from_tracks(tc_tracks, centroids, pool=None)
    tc_haz.check()
    path = os.path.join(path, "historical")
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    file_name = OUT_FILE_NAME.format(basin=basin)
    tc_haz.write_hdf5(os.path.join(path, file_name))

for basin in ["NI", "SI", "NA", "SP", "WP", "SA", "EP"]:
    main(centroids)

In [6]:
import pandas as pd
import numpy as np
from climada.hazard import Hazard
import xarray as xr
from pathlib import Path
haz = []
dir = DATA_DIR / "tropical_cyclones/historical"
for basin in ['EP','NA','NI','SA','SI','SP','WP']:
    haz.append(Hazard.from_hdf5(dir / f'tropical_cyclone_3600arcsec_genesis_{basin}_2000_2022.hdf5'))
tc_hazard = Hazard.concat(haz)


In [ ]:
len(tc_hazard.date)

In [16]:
import datetime
import numpy as np
import xarray as xr

# Convert ordinal dates to datetime
ordinal_dates = tc_hazard.date
dates = np.array([datetime.date.fromordinal(d) for d in ordinal_dates])

lat_lon_array = tc_hazard.centroids.coord
sparse_data = tc_hazard.intensity
data = sparse_data.toarray()  # Convert sparse matrix to dense if necessary

# Extract latitudes and longitudes
latitudes = np.unique(lat_lon_array[:, 0])
longitudes = np.unique(lat_lon_array[:, 1])

# Create indices for latitudes and longitudes
lat_indices = np.searchsorted(latitudes, lat_lon_array[:, 0])
lon_indices = np.searchsorted(longitudes, lat_lon_array[:, 1])

# Create the 3D array with the correct shape
reshaped_data = np.zeros((len(dates), len(latitudes), len(longitudes)))

# Use NumPy advanced indexing to populate the 3D array
reshaped_data[:, lat_indices, lon_indices] = data

# Create a DataArray with dimensions (time, latitude, longitude)
dates = pd.to_datetime(dates)

data_array = xr.DataArray(reshaped_data, coords=[dates, latitudes, longitudes], dims=['time', 'latitude', 'longitude'])

# Create a dataset
ds = xr.Dataset({'data': data_array})



In [17]:
ds = ds.groupby('time').sum()


In [18]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 572, longitude: 1406, time: 1930)
Coordinates:
  * latitude   (latitude) float64 -67.25 -67.0 -66.75 -66.5 ... 75.0 75.25 75.5
  * longitude  (longitude) float64 -179.8 -179.5 -179.2 ... 179.5 179.8 180.0
  * time       (time) datetime64[ns] 2000-01-03 2000-01-06 ... 2022-02-09
Data variables:
    data       (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [20]:
monthly_max = ds.resample(time='M').max()

# Create a complete month range covering the entire period
start_date = pd.to_datetime(dates.min()).to_period('M').start_time
end_date = pd.to_datetime(dates.max()).to_period('M').end_time
complete_months = pd.date_range(start=start_date, end=end_date, freq='M')

# Reindex to include all months in the range, filling missing values with 0
monthly_max = monthly_max.reindex({'time': complete_months}, fill_value=0)


In [25]:
output_dir = DATA_DIR / 'tropical_cyclones/max_monthly'
os.makedirs(output_dir, exist_ok=True)

# Save one file per year
for year in np.unique(monthly_max['time.year']):
    yearly_data = monthly_max.sel(time=str(year))
    output_path = os.path.join(output_dir, f'tropical_cyclones_max_monthly_{year}.nc')
    yearly_data.to_netcdf(output_path)

In [ ]:
yearly_hazards[year].intensity.sum()

In [ ]:
ds.data.shape